[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aynetdia/Budget_Constrained_Bidding/blob/master/Tutorial.ipynb) 

# Setting up the notebook in Colab

In [ ]:
# Mount on GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cloning the repo (only when setting up for the first time)
%cd drive/MyDrive
!git clone https://github.com/aynetdia/budget_constrained_bidding.git

Here's a link to the dataset: https://drive.google.com/drive/folders/1YYyxGMDW0EuZA2BI-uR_2j60lpngKgy8?usp=sharing

In order to set up the dataset, you either have to download the linked folder containing the dataset and put it into `/budget_constrained_biddig/data/ipinyou/`, in case you choose to run the notebook locally, or add a GDrive shortcut (go to: Shared with me -> Right click on the dataset folder -> Add shortcut to Drive) and select the  same `/budget_constrained_biddig/data/ipinyou/` as a location to place the shortcut in, if you want to run the notebook in Colab.

Link to the Github repo: https://github.com/aynetdia/budget_constrained_bidding.git

If needed, all the package requirements are listed in the requirements_pip.txt (for pip envs) and requirements_conda.txt (for conda envs) files, contained inside the Github repo.

In [ ]:
# Go into the project folder and pull the last changes (do that every time before running the notebook)
# If executing after cloning the repo: %cd budget_constrained_bidding
%cd /content/drive/MyDrive/budget_constrained_bidding
!git pull

# Table of Contents

&emsp;&emsp;1. Introduction into the application domain

&emsp;&emsp;2. Methods

&emsp;&emsp;3. DRLB Framework

&emsp;&emsp;4. Description of the Data Set

&emsp;&emsp;5. DRLB Implementation

&emsp;&emsp;6. DRLB Results 

&emsp;&emsp;7. References

# 1. Introduction

Allocating budget efficiently is a problem that has received a lot of attention in marketing literature.
Budget constrained bidding, as an automated bidding strategy and a common scenario in Real-Time Bidding (RTB), is an important concept in online display advertising. In the context of online marketing strategies, budget constrained bidding serves as an important mechanism in which a typical optimization goal for advertisers is to maximize the total value of winning impressions under a certain budget constraint. A winning impression value is usually associated with the expected outcome of an advertisement, e.g. a click or conversion as user feedback. In RTB advertisers give a bid for every individual impression of their advertisements instead of settling for a predefined fixed bid for each campaign. RTB enables advertisers to spend campaigns budget on high-quality ad impressions in order to obtain that positive user feedback. Thus, one challenge is to capture those valuable ad impressions in real-time, which can be solved by an appropriate bidding strategy.
    




### Budget Constrained Bidding

A bidding strategy is based on the evaluation of impressions on an ad campaign, e.g. by the predicted click-through rate (CTR) or calculated bid price for each impression. A campaign runs during a certain time period that captures $N$ impression opportunities in sequential order $i$. Considering the competition with other bidders in real-time and the impression value $v{_i}$, an advertiser gives a bid $b{_i}$ and only wins the auction if $b{_i}$ is the highest bid (Wu D., et al., 2018). In this case the advertiser gets the opportunity to display the ad and to gain winning impressions which then induces costs $c{_i}$.
However, budget constrained bidding is basically a knapsack problem (Zhou Y., et al., 2008). An optimal bid in RTB with budget constraint B takes the from of:

$$ b{_i} = v{_i} / \lambda, $$

where $v{_i}$ denotes the impression value of an ad placement and $\lambda$ a scaling parameter mapping $v{_i}$ to the scale of the bid (Zhang W., et al., 2016). If $\lambda$ increases, then the bid price decreases. Hence, high constrained budget should cause the general bidding price level to be lower (Zhang W., et al., 2014).
Bidding strategies usually consider the bid decision by treating each ad impression independently, ignoring the impacts of the overall effectiveness of the campaign and the dynamics of the RTB environment. Therefore, developing an optimal bidding strategy is complex to derive but yet highly important since in RTB only the advertiser with the highest bid wins the auction. Reinforcement Learning (RL) is a solution for the above problem.  


This tutorial presents a solution for the budget constrained bidding problem, which leverages model-free reinforcement learning. In this model-free reinforcement learning, state spaces are the impressions feature parameter as well as the auction information and the action is to set a bidding price. The tutorial is based on the findings of Wu D., et al. (2018) presented in \"Budget Constrained Bidding by Model-free Reinforcement Learning in Display Advertising\".
In this tutorial, we first introduce basic concepts of Reinforcement Learning, Constrained Markov Decision Process and Deep Q-Networks. In the next section we explain the Deep Reinforcement Learning to Bid framework, followed by a brief description of the dataset and an explanation on the implementation of the algorithms. Lastly, we discuss the results. 
 

# 2. Methods

## 2.1 Introduction to Reinforcement Learning and Machine Learning paradigm comparison

### Knapsack problem
	
The basic idea of the Knapsack problem is to find an optimal strategy under some constraints. For example, how to choose the most suitable items to be placed in the backpack in order to maximize the values of the items, given a limited total weight. Such problems can usually be solved recursively, i.e., by converting the original problem into an iterable subproblem, and then solving it optimally by dynamic programming methods. KP is decomposed into multiple P, each instance P’ in RL represents a P and will be described as [ state,action,reward ] (Reza R.A, Yingqian Z., Murat F. et al， 2020).




### Reinforcement Learning  
Reinforcement Learning (RL) is a tool to support sequential decision-making events and is a highly active research field of machine learning. It finds enormous application in e.g. real-time-bidding in advertisement auction.

RL is a core control problem in which an agent sequentially interacts with an unknown environment to maximize its cumulative reward (Richard S. and Andrew G., 2018). It is a simulation of the human learning process in dynamic environments without any supervision. To find the best strategy of reward maximization, the agent continuously interacts with the environment and finds the optimal action under different states. Roughly speaking, the agent’s goal is to get as much reward as possible over the long run.

For instance, board game playing is a classical dynamic decision-making process. Players need to interact with their counterparts. It is a procedure filled with immediate rewards, intuitive judgments, and each action will generate an impact in the end. The cumulative reward will decide its win-or-failure (Richard S. and Andrew G., 2018).

In reality, such scenarios will consist of several problems that can be solved by a class of solution methods, but not all of them can achieve the rewards maximization objective. In the learning process of RL, each action the agent made, will generate some impact not only on the immediate reward but also on the future states. In other words, RL faces a dynamic decision-making problem, the agent will obtain continuous rewards and punishments in this learning process and modify its behaviors according to the environment's feedbacks and finally to maximize the cumulative reward.

In order to make the process more clear, we explain some basic concept of RL as follows:

- Policy: the whole actions the learning agent has taken in a concrete period 

- Agent: the one who takes the action

- Environment: the place where the agent takes action and interacts with him

- Action: the move the agent makes to interact with the environment

- State: a situation in which the agent perceives 

- Reward: feedback of the agent’s action 	

And the following image illustrates how typically agents interact with their environments in RL-based models:

![Image 1: Reinforcement Learning Process](https://drive.google.com/uc?export=view&id=1y_z3Xg_QCsK9kkfV3tWhtqJNDmW4PrAJ)

	


### ML paradigm comparison	
Besides RL, supervised learning and unsupervised learning are the most classic machine learning methods. In this section, we explain why RL is the best choice for solving dynamic decision-making problems. Thus, the core differences among these three methods are shown in table below.


![Table 1: Machine Learning Methods Comparison](https://drive.google.com/uc?export=view&id=1U8oQjgdYPhFZ-FMyuyF2MPYSIuXksVkq)

Supervised learning is mainly used to solve regression and classification problems. It explores the relation among labeled, the target variable, and other input variables and produces a model to predict further category identification.

In comparison, unsupervised learning obtains unlabeled observations and searches the hidden structure behind the input data. Both of these methods are not suitable for the desired behavior optimization problem in a dynamic process. Unlike supervised and unsupervised learning, which will not consider the capacity or some constrained environments, reinforcement learning will start with a clear initial setting, goal, and interactive agent. All the sequential action the agent takes will generate some influence in this environment. After a dynamic model training process, the agent obtains more and more rewards as desired by taking optimal actions in order to maximize the cumulative reward. 

Reinforcement Learning models heavily lean on the concept of a Markov Decision Process, that is used to formulate a given problem and thus find an optimal solution. Generally, when we can map our problems into an MDP, we can expect that reinforcement learning can be a useful tool to solve such problems. In the next section, we will explain the details behind Markov Decision Processes and how they are constructed, and give more details about Deep Q-Learning, which is a core concept in our project.


## 2.2 (Constrained) Markov Decision Process  
  
### Markov Decision Process 
MDP is used to determine how good an agent's actions $a$ $\in$ $A$ in a particular state $s$ $\in$ $S$ is. This is determined by a policy (Singh A., 2019):

$$\pi: S \rightarrow P(A).$$ 

The policy is a function that defines the probability distribution over actions in each state, which fully determines the behavior of an agent. The agent uses the policy to interact with the environment by sequentially taking actions and to alter its behavior in order to maximize the cumulative reward $R$ (all the rewards that the agent receives from the environment) (Wu D., et al., 2018). In MDP the state transition $T(s, a, s^\prime)$ produces the probability $Pr(s^\prime | s, a)$ to transition into state $s^\prime$ given that the agent was in state $s$ and took action $a$.  The immediate reward function, which is the reward $r$ an agent receives from the current state, shows the agents immediate reward from the environment. If an agent follows a policy $\pi$  at time $t$  then $\pi(a | s)$ is the probability of that agent taking action $a$ at a particular time step $t$ (Singh A., 2019).

The goal of the agent is to learn the optimal policy $\pi$  which maximizes the long-term expected reward form the start state (Wu D., et al., 2018): 

$$\pi^* = arg max_{\pi}  E [R | \pi],$$ where $R = \sum^T_{t=1} \gamma^{t-1} r_t $ denotes the cumulative discounted reward.  



### Constrained Markov Decision Process

Constrained Markov Decision Process (CMDP) (Altman E., 1999) extends the concept of MDP by adding additional constraints, e.g. a cost constraint. 

While learning in an unknown CMDP, an agent should trade-off exploration and exploitation. Accordingly, the agent explores to discover new information about the MDP, and exploits on the current knowledge to maximize the reward while satisfying the constraints.In an episodic CMDP the agent regulates $\lambda$ with a fixed time step T until the episode ends. The agent observes states and takes action to adjust $\lambda_{t-1}$ to $\lambda_{t}$ during each time step $t$. However, the agent will still converge to the suboptimal solution due to the immediate reward function (Wu D., et al., 2018). 
 
Note that the (C)MDP is often solve using model-based reinforcement learning approaches, where the state transition dynamics $T$ needs to be known in advance but is yet hard to predict, when considering the dynamic and unpredictable environment of an agent. Besides, model-based approaches suffer from the scalability problem, since calculations of state transitions matrices, which are then solved using dynamic programming algorithms, require high computational costs, which may be detrimential when deployed in real-world scenarios.

In contrast to this, model-free reinforcement learning approaches solves reinforcement learning tasks directly by solely using the policy function. Therefore, our focal paper proposes to use deep Q-Networks (DQN), which is a model-free RL approach. 

### Bellman Equation for value functions

The Bellman Equation helps to determine optimal policies and optimal value functions. The agent chooses the action according to the imposed policy. Different policies implies different value functions for the state. The state value function expresses the relationship between the value of a state $s$ and the values of its successor states $s^\prime$ . The value of a state under an optimal policy must equal the expected reward for the best action from that state. It is decomposed into the immediate reward, $R[t+1]$ , and the discounted value of successor states (Singh A., 2019):

$$v(s)=E[R_{t+1}+\gamma+v(S_{t+1}) | S{_t} = s].$$  





## 2.3 Deep Q-Networks


Deep Q-Network (DQN) (Mnih V, Kavukcuoglu K., Silver D., et al., 2015) is a popular method of Q-learning. It achieves impressive results in a variety of challenging tasks - e.g. it succeeded at achieveing performance comparable to that of a professional human games tester across a set of 49 games. DQN combines the methods of reinforcement learning and deep neural networks to map the environment states to Agent actions. The goal of an agent is to select actions in order to maximize the cumulative future reward. DQN uses deep neural networks to approximate the action-value function (Q-function) corresponding to the optimal policy without the need of handcrafting useful features. The optimal action-value function $Q^*$ is defined as the maximum sum of rewards $r_{t}$ discounted by $\gamma$ at each tilmestep $t$. This is achieved by the behavior policy $\pi = P(a|s)$ of the agent. It defines the value of taking action $a$ in state $s$ under policy $\pi$:

$$ Q^*(s,a) = max_\pi E[r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... | s_t=s , a_t = a, \pi ].$$

Reinforcement learning is unstable when using non-linear function approximators (neural networks) to represent the action-value function. Reasons of these instabilities are that small updates of the Q-value may significantly change the policy and thus, change the correlation between action-values ( $Q$ ) and the target values ( $ r + \gamma max_{a'}Q(s',a')$ ). The correlation presents the sequence of states. DQN uses target networks to cope with these instability issues (Stekolshchik R., 2020). The target network is a technique to stabilize learning by delaying the update of target values. It is a copy of local network which is held as a constant in order to stable the target for learning for a fixed number of $T$ time steps.
Also, DQN samples random minibatches of transition from experience replay buffer, which stores data into specific memory as training data, in order to reduce correlations.

In DQN the parameter $\epsilon$ of the $\epsilon$-greedy mechanism controls the proportion of exploration and exploitation, where $\epsilon$ is bounded at 0 < $\epsilon$ < 1. The exploitation describes that the agent finds his following action by maximizing the Q-value over all possible actions for the given state.

All in all, the agent uses deep neural networks to determine its actions and to train the neural network in such a way that it predicts the weighted, cumulative rewards of all actions. In DQN the agent learns the optimal policy by interacting directly with the workflow environment and also with the policies of other agents.


# 3. Deep Reinforcement Learning to Bid (DRLB)

To address the problem of budget constrained bidding in real-time advertisement, Wu et al. (2018) proposed a framework that is built on top of DQN - Deep Reinforcement Learning to Bid (DRLB). The advantage of this framework for RTB is given by the fact that it is a model-free RL approach.

Thus, Wu et al. proposed training a **DQN agent** that sequentially regulates the bidding parameter $\lambda$ (remember, the bid in DRLB takes form of $b_i = v_i/{\lambda}$) instead of directly producing bids, which is the case in model-based approaches like the one introduced by Cai et al. (2017). Furthermore, instead of using the immediate reward from the environment to determine an optimal policy, an additional deep neural network (called **RewardNet**) is trained simultaneously with the DQN to predict the reward. Wu et al. (2018) found that simply using the immediate reward will make the agent obsessed with taking actions to agressively decrease $\lambda$ from the get ho to gain more immediate reward at each step and thus effectively neglect the budget constraint.

Additionally an **adaptive $\epsilon$-greedy policy**, that adjusts the exploration probability based on how the state-action value is distributed, was designed for DRLB. Hence the agent chooses action (adjusting the $\lambda$ parameter):

$$ a^* = argmax_a Q(s,a) $$

with probability $1 -\epsilon$ or otherwise takes a random action. $\epsilon$ is usually initialized with a large value and then gradually annealed to a small value over time. This balances the exploration-exploitation trade-off during training, by encouraging the agent to take more random actions in the beginning of the training process (and thus explore a variety of actions), and rely on the learned DQN parameters (exploit them) further down the training process. 

Before delving more into details of DRLB implementation, we'll describe the dataset that was used to train and evaluate the DRLB.

# 4. Description of the Data Set

This section gives a brief insight into the data structure and depicts the results of some explanatory data analysis (EDA) to provide a better understanding of the data.    

The datasets are provided by the Demand-Side Plateform (DSP) iPinYou Information Technologies Co., Ltd which is an established leading provider for audience-based programmatic adverting technology and builds RTB technology and algorithms. We examine the dataset of the advertiser with the advertiser ID "1458" that operates in the chinese vertical e-commerce industry. 

The data sets are split into train and test sets according according to Zhang et al. (2014), using their [Github repository](https://github.com/wnzhang/make-ipinyou-data), which is considered a benchmark format of the iPinYou dataset and also used for training and evaluating the DRLB framework. For the train set the first seven days are used for training, while for the test dataset the last three days are used for evaluation. Hence, in total the datasets comprise tracked performance from ten continous days.

Both data sets generally record the auction price (i.e. paying and bidding price), user feedback (i.e click, conversion, impression) and ad features (e.g. ad slot ID, width, height, visibility, floor price). In this dataset we deal with the **second price auction**, which means that the varaible *payprice*, representing the highest bid of "opponents" bidding simuiltaneously on a given ad placement, determines how much in the end will be paid for the bid request.

Another important aspect is that we slightly modified each dataset by including additional variables, in accordance with Wu et al (2018). We added the predicted click-trough-rate (pCTR) and time intervals for every quarter of an hour in minutes (min). The pCTR is used as an indicator of the impression value (i.e. how much is a given ad placement worth - the higher the probability of the click, the more it is worth to the advertiser), which is a second parameter ($v_i$) used to calculate the bid amount for a given ad placement, as explained in the introduction. Each 15 min interval represents a time step in the DRLB framework.

As for the variable pCTR, we applied a logistic regression to predict the CTR based on the features of the train set and clicks as our target variable.  

In [ ]:
#you should find yourself inside the project folder after you've pulled the latest changes
!ls

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load datasets
df_bid_train = pd.read_csv('data/ipinyou/1458/train.log.txt', sep="\t") 
df_bid_test = pd.read_csv('data/ipinyou/1458/test.log.txt', sep="\t") 

In [ ]:
# This is an overiew of the fratures in the training data
df_bid_train.info()

In [ ]:
# View dimension and missing values of the train set
#print(df_bid_train.columns)
print('Dimensionality of the train data is {}'.format(df_bid_train.shape)) 
print('The data set has {} cases.'.format(df_bid_train.shape[0]))    
print('The total number of elements is {}.'.format(df_bid_train.size))
print('The total numbers of missing values of each variable in train data are \n{}.'.format(df_bid_train.isna().sum()))

In [ ]:
#'This is the feature description of the test data.'
df_bid_test.info()

## Explanatory data analysis

In [ ]:
# Frequency of clicks
df_bid_train.click.value_counts()

In [ ]:
# # amount of impressions bases on regions
df_bid_train.region.value_counts().plot(kind='bar') 
plt.ylabel('amount of clicks')
plt.xlabel('regioh')
plt.show()

In [ ]:
# amount of impressions group by hours
df_bid_train.hour.value_counts().plot(kind='bar') 
plt.ylabel('amount of impressions')
plt.xlabel('hours')
plt.show()

In [ ]:
## The amount of impressions based on the useragents levels of browser types and operation system types 

In [ ]:
# best performing slotwidth 
df_bid_train[df_bid_train['click']==1].slotwidth.value_counts().plot(kind='bar') 
plt.ylabel('amount of clicks')
plt.xlabel('ad slot width')
plt.show()

In [ ]:
# best performing slotheight 

df_bid_train[df_bid_train['click']==1].slotheight.value_counts().plot(kind='bar') 
plt.ylabel('amount of clicks')
plt.xlabel('ad slot height')
plt.show()

In [ ]:
# best performing slotvisibility 

df_bid_train[df_bid_train['click']==1].slotvisibility.value_counts().plot(kind='bar') 
plt.ylabel('amount of clicks')
plt.xlabel('ad slot visibility')
plt.show()

# slotvisibility:
#0 equals unknown
#1 equals First View above the fold
#2 equals Second View beneath the fold


In [ ]:
# distribution of payprice

df_bid_train.payprice.plot(kind='box', showfliers=False) 

In [ ]:
# the average costs: CPC (if clicked), CPM (if not clicked) , CTR () , CVR , CPConv
CPC_mean=df_bid_train[df_bid_train['click']==1].payprice.mean()
CPM_mean=df_bid_train[df_bid_train['click']==0].payprice.mean()
print("the mean payprice for click is:{}\nthe mean payprice for unclick is: {} ".format(CPC_mean,CPM_mean))

## Preprocessing the dataset for logistic regression

The following briefly explains the preprocessing of the train data to obtain the predicted CTRs which is based on the click of the train set as our target variable.   

To estimate the CTR and therefore to predict the probability of a user to click on an ad impression, we apply the logistic regression model as our prediction model. In the logistic regression models most of our features are binary. Thus, most non-binary features we converted into categorical features first and then transformed them into dummy variables.

Also, we briefly want to highlight some of the (binary) feature transformations more in detail. From user agent we exctracted the browser and operation systems. User tags are processed to extract the tag list of each user. According to Zhang et al. (2014) the weekday and hour feature are extracted from the timestamps. As mentioned earlier, we additionally extracted time intervals for every quarter points of an hour in minutes. Furthermore, region, city, ad slot width, height, visibility, format, ad exchange are processed in the logistic regression model as binary variables and the ad slot floor price as numeric. Overall, we included binary features, which are quite generic and not fixed or unique values such as IDs, URLs or paying price. In total, we have 312 binary features for the logistic regression model. 

In the logistic regression model the default of L2 regularization is used. Also, we want to emphasize that a significant imbalance of 0 (no clicks) and 1 (clicks) values exists in the target variable. Hence, we balanced the minority class 1 by using the class_weight parameter of the logistic regression function.

In [ ]:
from sklearn.metrics import roc_auc_score
print("pCTR AUC in the train set: ", roc_auc_score(df_bid_train['click'], df_bid_train['pCTR']))
print("pCTR AUC in the test set: ", roc_auc_score(df_bid_test['click'], df_bid_test['pCTR']))

Surprisingly, our prediction model yields a very high prediction accuracy, and especially interesting is the fact the AUC in the test set is higher than in the train set. We double-checked our logistic regression and implementation and couldnt't find any data leakage from the test set or the target into the fitted model.

In [ ]:
##training Logitstic Regression model and predict the CTR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import joblib


class Logistic_Regression():
    
    # The dummified datasets is not proviuded, due to their big size (>5GB)
    df_bid_requests=pd.read_csv(".../dummified_train_test.txt", sep="\t")
    df_bid_test=pd.read_csv("...//dummified_test_test.txt", sep="\t")
    
    #training

    df_bid_test=pd.DataFrame(df_bid_test.astype(np.uint8))
    df_bid_requests=pd.DataFrame(df_bid_requests.astype(np.uint8))

    X = df_bid_requests.drop(['click','slotprice'], axis=1)
    Y = df_bid_requests.click
    df_bid_test=df_bid_test.drop(['slotprice'],axis=1)

    #df_bid_test = df_bid_test[np.intersect1d(X.columns, df_bid_test.columns)]

    X=X[np.intersect1d(X.columns, df_bid_test.columns)]

############ Train model with Logitstic Regression
    #Logistic Regression: Model fitting
    
    logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
    logreg.fit(X, Y)

    #save models
    joblib_filename = "logreg.sav"
    joblib.dump(logreg, joblib_filename)


    # Check auccuracy score
    print(logreg.score(X,Y))
    prob = logreg.predict_proba(X)[:, 1]
    print(roc_auc_score(Y, prob))

    logreg = joblib.load("logreg.sav")
    #
    # # ## Logistic Regression: Predict CTR
    ctr_pre_train=logreg.predict(X)
    ctr_pre_train_proba=logreg.predict_proba(X)
    df_ctr_train = pd.DataFrame(ctr_pre_train_proba, columns=['ctr_prediction_pro','ctr_prediction'])
    df_ctr_train['click'] =ctr_pre_train
    df_ctr_train.to_csv('.../ctr_pred_train.txt', index=False, sep='\t', header=True)

    ctr_pred = logreg.predict(df_bid_test[X.columns])
    ctr_pre_proba = logreg.predict_proba(df_bid_test[X.columns])


    df_ctr_test = pd.DataFrame(ctr_pre_proba, columns=['ctr_prediction_pro_0', 'ctr_prediction_pro_1'])
    df_ctr_test['click']=ctr_pred
    print(df_bid_test)
    df_ctr_test.to_csv('.../ctr_pred_test.txt',
                         index=False, sep='\t', header=True)

In [ ]:
# add the min intervals

def get_time_interval(data):
    time_inv=int(data[10:12])
    if time_inv>=0 and time_inv<15:
            return ("00")
    elif time_inv >= 15 and time_inv < 30:
            return ("15")
    elif time_inv >= 30 and time_inv < 45:
            return ("30")
    elif time_inv >= 45 and time_inv <=60:
            return ("45")
    else:
            return(None)

if 'min' in bid_requests.columns:
    print("min already exists")
    pass
else:
    bid_requests["timestamp"]=bid_requests["timestamp"].apply(str)
    min_intervals = bid_requests.apply(lambda row : get_time_interval(row['timestamp']), axis = 1)
    bid_requests.insert(3, "min", min_intervals) # insert the new column after the 'hour' column
    # save the updated dataset. do not run again!
    # bid_requests.to_csv('data/ipinyou/1458/train.log.txt', sep="\t", header=True, index=False)

# 5. DRLB Implementation

Since the implementation of DRLB was not made public by the authors, we had to code it on our own. Although we've found [one implementation](https://github.com/venkatacrc/Budget_Constrained_Bidding) of DRLB on Github, it is missing some functionality that was introduced in the paper, and, to be honest, has some parts that are flat out wrongly implemented. Therefore, even though this code was very useful as a skeleton to base our implementation on, we've added a lot of fixes and improvements to it. 

Since the codebase is quite large to use directly in the notebook, we'll include here only some important parts. Let's first start by looking at how each state $s_t$ state in the (episodic) CMDP formulated for the RTB problem is defined. Remember, one episode is represented by one day in the iPinYou data, while one time step is equal to a 15 min interval:

In [ ]:
def _get_state(self):
    """
    Returns the state that will be used as input in the DQN
    """
    return np.asarray([self.t_step, # 1. Current time step
            self.rem_budget, # 2. the remaining budget at time step t
            self.ROL, # 3. The number of Lambda regulation opportunities left in the episode (amount of time steps left)
            self.BCR, # 4. Budget consumption rate: (Bt - Bt-1)/Bt
            self.CPM, # 5. Cost per mille of impressions between t-1 and t: (costs at t/impressions at t)*1000
            self.WR, # 6. Auction win rate at state t: (won impressions at t)/(total bidding opportunities at t)
            self.rewards_prev_t]) # 7. Total clicks on the ad placements we've won at t-1

Next, let's see how the bidding agent that acts in our problem's environment, based on the states that it receives, is initialized:

In [ ]:
class RlBidAgent():
    
    def __init__(self):
        # Beta parameter adjusting lambda, that regulates the agent's bid amount
        self.BETA = [-0.08, -0.03, -0.01, 0, 0.01, 0.03, 0.08]
        # Starting value of epsilon in the adaptive epsilon-greedy policy
        self.eps = 0.9
        # Parameter controlling the annealing speed of epsilon, suggested by Wu et al. (2018)
        self.anneal = 2e-5
        # DQN Network to learn Q function
        self.dqn_agent = DQN(state_size = 7, action_size = len(self.BETA))
        # Reward Network to learn the reward function
        self.reward_net = RewardNet(state_action_size = len(self.BETA) + 1, reward_size = 1)
        # Number of timesteps in each episode (4 15min intervals x 24 hours = 96), as formulated by Wu et al. (2018)
        self.T = 96
        # Initialize the DQN action for t=0 (index 3 - no adjustment of lambda, 0 in self.BETA)
        self.dqn_action = 3

The first beta parameter is basically all the actions that the DQN is able to take at each time step to control the lambda parameter, thus corresponding to the action size (i.e. the output layer) of the DQN. It outputs the distribution of Q values over all actions, so e.g. if the highest Q value is at index 3 of the output layer, the bidding agent adjusts lambda by 0 percent:

$$ \lambda_{t-1} = \lambda_t \times (1 + \beta_a). $$

The RewardNet takes the state-action pair as input, while outputting one reward value. Both networks consist of 3 hidden layers with 100 neurons in each layer and ReLU activation functions. Here's how each of the networks is initialized:

In [ ]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 32         # minibatch size
GAMMA = 1.0             # discount factor
LR = 1e-3               # learning rate 
C = 100                 # how often to update the network

class DQN():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.

            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        set_seed()

        # Q-Network
        self.qnetwork_local = Network(state_size, action_size).to(device)
        self.qnetwork_target = Network(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)
        self.criterion = nn.MSELoss()

        # Replay memory
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, 0)

The reason for using two differnet instantiations of the Q network simultaneously was explained by Minh et al. (2015), who introduced the DQN itself. Every $C$ updates the `qnetwork_local` is cloned to obtain `qnetwork_target`, which is used for generating the Q-learning targets $y$ for the following $C$ updates to `qnetwork_local`. This makes the algorithm more stable, since generating the targets using an older set of parameters adds a delay between the time an update to `qnetwork_local` is made and the time the update affects the targets $y$, making divergence or oscillations of the learned policy much more unlikely. The input to the `qnetwork_local` is the current state $s_t$, while the `qnetwork_target` receives state $s_{t+1}$ as input.

To get an intuitive understanding for why the targets for predictions of the DQN for $s_t$ are predictions of (almost) the same DQN for $s_{t+1}$, think about it as a way to peek into the future using all available resources at the current time step. Maybe knowing how the DQN performs an update of its parameters will aid in understanding this idea a bit better:

In [ ]:
def learn(self, experiences, gamma, terminal):
        """Update value parameters using given batch of experience tuples.

            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s') tuples 
            gamma (float): discount factor, always equal to 1 in DRLB
        """
        states, actions, rewards, next_states = experiences

        if terminal == 1: # if the next state is the last one in the episode
            y = rewards
        else:
            y = rewards + gamma * self.qnetwork_target(next_states).max(1, keepdim=True)[0] # calculate best actions for each of the 32 s_t+1

        # Get Q values from local model
        Q_local = self.qnetwork_local(states).gather(1, actions) # get the Q values of the current DQN for the hsitorical actions taken

        # Compute loss
        loss = self.criterion(Q_local, y)
        print("DQN loss = {}".format(loss))
        # Grad descent
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.loss = loss.item()
        # Every C steps reset Q target = Q (hard copy)
        if ((self.t_step + 1) % C) == 0:
            for target_param, local_param in zip(self.qnetwork_target.parameters(), self.qnetwork_local.parameters()):
                target_param.data.copy_(local_param.data)

Every $C=100$ steps the DQN samples a mini-batch containing 32 of each (corresponding to each other) $s_t$, $s_{t+1}$, $r_t$ and $a_t$ from its replay memory (initialzied above with maximum length of 100 000 records/states). So, the action of DQN in state $s_t$ is evaluated based on the action that it would take at the next step $s_{t+1}$, based on the reward for the action at the current step. And this chain can be unfolded for each time step, except for the last one in the episode. 

The gamma parameter is always set to 1 in DRLB, since the optimization goal of the budget constrained bidding problem is to maximize the total reward value under the cost constraint regardless of the reward time (i.e. we don't want to punnish getting clicks on the won ad placements later in the episode).

But how does the DQN decide on which actions to make?

In [ ]:
def act(self, state, eps, eval_flag):
        """Returns actions for given state as per current policy.

            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)[0] # [0] 'cause otherwise nested array
        self.qnetwork_local.train()

        if eval_flag == 0:
            # Epsilon-greedy action selection
            # Check if the Q-value distribution is unimodal, if so:
            if self.unimodal_check(action_values) == True:
                if random.random() <= eps:
                    # choose action randomly with prob epsilon
                    return random.choice(np.arange(self.action_size))
                else: # and a regular action with 1-eps
                    return np.argmax(action_values.cpu().data.numpy())
            # If not unimodal, increase epsilon, if it's small
            else:
                prob = max(eps, 0.5)
                if random.random() <= prob:
                    return random.choice(np.arange(self.action_size))
                else: # and with 1-p choose an action regularly 
                    return np.argmax(action_values.cpu().data.numpy())
        else:
            return np.argmax(action_values.cpu().data.numpy())

The unimodality check of the action-value distribution $Q(s_t, a_t)$, was proposed by the authors of DRLB - theyu believe that when the distribution is not unimodal, the current estimation of Q is abnormal and the $\epsilon$ should be increased to encourage more explorations under this state. The following figure taken from the DRLB paper illustrates the unimodal (normal) vs abnormal distribution. The unimodality check is done using a simple heuristic test.

<img src="https://drive.google.com/uc?export=view&id=1jtIbMzTW6AHQDhwoW5kyDRARu4EsBbEh" width=750 height=750 />

Below are the full training algorithms of the DQN and the RewardNet. The training process of the RewardNet is much simpler, since it simply learns to predict an optimal reward for a given state-action pair.

<img src="https://drive.google.com/uc?export=view&id=1Eq4-jLRZ054LRgRy5eUv7Bkk5xuFAqCn" width=900 height=900 />


The $\lambda_0$ at the beginning of each episode is calculated by solving the bidding problem as a knapsack problem using the greedy approximation algorithm introduced by Dantzig (1957).

In [ ]:
def calc_greedy(items, budget_limit):
    # Borrowed from: https://bitbucket.org/trebsirk/algorithms/src/master/knapsack.py
    bids = []
    spending = 0
    ctr = 0
    items_sorted = sorted(items, key=itemgetter(2), reverse=True) # sort bid requests by pCTR/payprice
    while len(items_sorted) > 0:
        item = items_sorted.pop()
        if item[0] + spending <= budget_limit:
            bids.append(item)
            spending += bids[-1][1]
            ctr += bids[-1][0]
        else:
            break
    ctrs = np.array(bids)[:,0]
    costs = np.array(bids)[:,1]
    # Take the max lambda to be more conservative at the beginning of a time step
    opt_lambda = np.max(np.divide(ctrs, costs))
    return opt_lambda

## Training DRLB

Since neither Wu et al. (2018), nor Cai et al. (2017), against whose framework the former compared DRLB, provide a budget constraint during training, we decided to initialize the budget for each episode as the total cost of the auction (sum of all payprices) divided by 4 and split into 7 equal parts. We thought that this may be a good balance between giving an agent enough money play around with, while also being cosntrained enough.

Warning: the DRLB takes a long time, therefore we commented out the training code, so you don't accidentally run this cell.

In [ ]:
# # Train DRLB

# import sys  
# import torch
# import cloudpickle
# sys.path.insert(0, 'src/rtb_agent')
# from rl_bid_agent import RlBidAgent
# from model import set_seed
# import gym, gym_auction_emulator

# #Instantiate the Environment and Agent
# env = gym.make('AuctionEmulator-v0')
# env.seed(0)
# set_seed()
# agent = RlBidAgent()

# epochs = 200

# for epoch in range(epochs):

#     print("Epoch: ", epoch+1)

#     obs, done = env.reset()
#     train_budget = env.bid_requests.payprice.sum()/4
#     # Set budgets for each episode
#     budget_proportions = []
#     for episode in env.bid_requests.weekday.unique():
#         budget_proportions.append(len(env.bid_requests[env.bid_requests.weekday == episode])/env.total_bids)
#     for i in range(len(budget_proportions)):
#         budget_proportions[i] = round(train_budget * budget_proportions[i])
#     agent.episode_budgets = budget_proportions
    
#     agent._reset_episode()
#     agent.cur_day = obs['weekday']
#     agent.cur_hour = obs['hour']
#     agent.cur_state = agent._get_state() # observe state s_0

#     while not done: # iterate through the whole dataset
#         bid = agent.act(obs) # Call agent action given each bid request from the env
#         next_obs, cur_reward, cur_cost, win, done = env.step(bid) # Get information from the environment based on the agent's action
#         agent._update_reward_cost(bid, cur_reward, cur_cost, win) # Agent receives reward and cost from the environment
#         obs = next_obs
#     print("Episode Result with Step={} Budget={} Spend={} wins={} rewards={}".format(agent.global_T, agent.budget, agent.budget_spent_e, agent.wins_e, agent.rewards_e))
#     agent.episode_memory.append([agent.budget, agent.budget_spent_e, agent.wins_e, agent.rewards_e])

#     # Saving models and history
#     if ((epoch + 1) % 10) == 0:
#         PATH = 'models/model_state_{}.tar'.format(epoch+1)
#         torch.save({'local_q_model': agent.dqn_agent.qnetwork_local.state_dict(),
#                     'target_q_model':agent.dqn_agent.qnetwork_target.state_dict(),
#                     'q_optimizer':agent.dqn_agent.optimizer.state_dict(),
#                     'rnet': agent.reward_net.reward_net.state_dict(),
#                     'rnet_optimizer': agent.reward_net.optimizer.state_dict()}, PATH)

#         f = open('models/rnet_memory_{}.txt'.format(epoch+1), "wb")
#         cloudpickle.dump(agent.dqn_agent.memory, f)
#         f.close()
#         f = open('models/rdqn_memory_{}.txt'.format(epoch+1), "wb")
#         cloudpickle.dump(agent.reward_net.memory, f)
#         f.close()

#         pd.DataFrame(agent.step_memory).to_csv('models/step_history_{}.csv'.format(epoch+1),header=None,index=False)
#         agent.step_memory=[]
#         pd.DataFrame(agent.episode_memory).to_csv('models/episode_history_{}.csv'.format(epoch+1),header=None,index=False)
#         agent.episode_memory=[]

#     print("EPOCH ENDED")

# env.close() # Close the environment when done

## Baseline: Random Bidding

As a baseline for our DRLB implementation we decided to first use just simple random uniform bidding. As a budget constraint Cai et al. (2017) provide the following calculation:

$$B = CPM_{train} \times 10^{-3} \times T \times c_0$$

where $CPM_{train}$ is the cost per mille of impressions in the training data, $T$ is the amount of steps in each episode and $c_0$ acts as the budget constraint parameter. Following their work, we set multiple values of $c_0$ = 1/32, 1/16, 1/8, 1/4, 1/2.

In [ ]:
#load the datasets
bid_requests_train = pd.read_csv('data/ipinyou/1458/train.log.txt', sep="\t")
bid_requests_test = pd.read_csv('data/ipinyou/1458/test.log.txt', sep="\t")

In [ ]:
# define budget for each episode according to (Cai et al., 2017)
CPM_train = 1000 * (bid_requests_train.payprice.sum()/len(bid_requests_train))
budget = CPM_train * (10**-3) * 96
episode_budgets = [budget/2,budget/4,budget/8,budget/16,budget/32]
bid_random = bid_requests_test
bid_budget = pd.DataFrame()
bid_budget['bugdet']=episode_budgets
bid_budget['portion']=['1/2','1/4','1/8','1/16','1/32']
bid_budget.round()

In [ ]:
# Random Bidding
import pandas as pd
import numpy as np
import random
random.seed(0)
np.random.seed(0)

# create a temporary dataframe
c_names=('click','day', 'slotprice', 'payprice','r_bid_amt','wins')
zero_Data=np.zeros(shape=(len(bid_requests_test),len(c_names)))
df=pd.DataFrame(zero_Data,columns=c_names)
df['day']=bid_requests_test['weekday']
df['click']=bid_requests_test['click']
df['slotprice'] = bid_requests_test['slotprice']
df['payprice'] = bid_requests_test['payprice']
# generate random bids for each bid request, no higher than the bids of other auction participants
df['r_bid_amt']=np.random.randint(0,bid_requests_test['payprice'].max(),len(bid_requests_test))

i = 1
for budget in episode_budgets:
    for day in df['day'].unique(): # for each episode
        episode = df[df['day'] == day].copy(deep = True) # get an episode subset from the whole test set
        rem_budget = budget # and set the budget
        while rem_budget > 0: # while the budget is not exhausted
            bid_request = episode.sample() # sample a random bid request
            if bid_request['r_bid_amt'].values[0] > bid_request['payprice'].values[0]:
                index = bid_request.index[0]
                episode = episode.drop(episode.loc[episode.index == index].index[0]) # remove bid request if we win the auction
                df.loc[index, 'wins'] = i # track which budget won the auction
                # adjust the budget accordingly
                rem_budget = rem_budget - max(bid_request['payprice'].values[0], bid_request['slotprice'].values[0])
                # stop if not possible to win anything anymore
                if rem_budget <= episode['payprice'].min():
                    rem_budget = 0
    i += 1

In [ ]:
rand_clicks = []
rand_imps = []
for i in range (4,7):
    for j in range(1,6):
        rand_clicks.append(df[(df['wins'] == j) & (df['day'] == i)]['click'].sum())
        rand_imps.append(len(df[(df['wins'] == j) & (df['day'] == i)]))

In [ ]:
# Display the results of random bidding

interables=[['clicks','impression'],['1/2','1/4','1/8','1/16','1/32']]
arrays=pd.MultiIndex.from_product(interables, names=['reward', 'portion'])
a=pd.Series(rand_clicks[:5] + rand_imps[:5])
b=pd.Series(rand_clicks[5:10] + rand_imps[5:10])
c=pd.Series(rand_clicks[10:] + rand_imps[10:])
display=pd.DataFrame(pd.concat([a,b,c],axis=1).values, index=arrays)
display.columns=["day 8","day 9","day 10"]
print("Results of random bidding on the test set:")
display

# 6. DRLB Results

Before we delve into the evaluation of the DRLB, we would like to note that we were not able to fully train the model for the neccesary amount of time steps. Wu et al. suggest at least 125.000 steps of training to get somewhat staisfiable results (ideally >250.000), while we were able to train only for a bit more than 25.000 steps. This was due to a discovery of mistake in the code on the day of the notebook submission. We still include the evaluation of the results of the DRLB with subpar parameters.

We will gladly upload the DRLB with properly trained parameters once we get our hands on them. We will simply upload the saved model to our Github repo, which will allow you to smoothly run the evaluation with the properly trained DRLB, if you wish to do so.

In [ ]:
import sys
import torch
import cloudpickle
sys.path.insert(0, 'src/rtb_agent')
from rl_bid_agent import RlBidAgent
from model import set_seed
import gym, gym_auction_emulator
import pandas as pd

env = gym.make('AuctionEmulator-v0')
env.seed(0)

bid_requests_train = pd.read_csv('data/ipinyou/1458/train.log.txt', sep="\t")

# Calculate lambda_0 for the first episode in the test set
last_train_episode = bid_requests_train[bid_requests_train['weekday'] == 6]
items = []
for obs in last_train_episode.values:
    items.append([obs[1], obs[25], obs[1]/max(obs[25], 1)])

# Set the budgets for each episode according to Cai et al. (2017)
CPM_train = 1000 * (bid_requests_train.payprice.sum()/len(bid_requests_train))
budget = CPM_train * (10**-3) * 96
eval_budgets = [budget/2,budget/4,budget/8,budget/16,budget/32]

for e_budget in eval_budgets:

    # Initialize the bidding agent
    set_seed()
    agent = RlBidAgent()

    # Load the saved models
    checkpoint = torch.load('models/model_state_40.tar')
    agent.dqn_agent.qnetwork_local.load_state_dict(checkpoint['local_q_model'])
    agent.dqn_agent.optimizer.load_state_dict(checkpoint['q_optimizer'])
    agent.reward_net.reward_net.load_state_dict(checkpoint['rnet'])
    agent.reward_net.optimizer.load_state_dict(checkpoint['rnet_optimizer'])

    # init the current budget for each episode
    agent.episode_budgets = [e_budget, e_budget, e_budget]

    # start evaluating
    obs, done = env.reset()
    agent._reset_episode()
    agent.ctl_lambda = agent.calc_greedy(items, e_budget)
    agent.cur_day = obs['weekday']
    agent.cur_hour = obs['hour']
    agent.cur_state = agent._get_state() # observe state s_0

    while not done: # iterate through the whole dataset
        bid = agent.eval_act(obs) # Call agent action given each bid request from the env
        next_obs, cur_reward, cur_cost, win, done = env.step(bid) # Get information from the environment based on the agent's action
        agent._update_reward_cost(bid, cur_reward, cur_cost, win) # Agent receives reward and cost from the environment
        obs = next_obs
    print("Episode Result with Step={} Budget={} Spend={} impressions={} clicks={}".format(agent.global_T, agent.budget, agent.budget_spent_e, agent.wins_e, agent.rewards_e))

env.close()

As expected, the performance of the not properly trained DRLB is unsatisfiable to say the least. Basically the agent spends all its money on the first bid request it sees (and obviously wins it). However this may not exactly stem from the fact that the model wasn't trained long enough. As a reminder, at the beginning of each episode the $\lambda_0$ parameter is set as the result of solving the knapsack problem of the previouis episode with the greedy aproximation algorithm. Even by taking the highest lambda produced by that algorithm (which leads to lower bids), that lambda is still much higher than the second highest bid on a given ad placement, which leads to a very high spending in the first episode.

To analyze this further, taking a look at the training history is necessary.

In [ ]:
# Import history of the last 10 epochs
history = pd.read_csv('models/step_history_40.csv', names=['global_T', 'rem_budget', 'ctl_lambda', 'eps', 
                                                           'dqn_action', 'dqn_agent_loss', 'rnet_r', 
                                                           'reward_net_loss'])
# Subset the last training episode
last_train_episode = history[history['global_T']>=26069].copy()
last_train_episode = last_train_episode.reset_index(drop=True)
last_train_episode

In [ ]:
# Plot the budget spending at the last episode of training
ax = last_train_episode['rem_budget'].head(50).plot()
ax.set_xlabel("Time step t")
ax.set_ylabel("Remaining budget at step t")

From the graph above it seems obvious that the starting $\lambda_0$ may be a bit too high, since there is a very sharp drop in the remaining budget in the first 10 steps (out of 96). We believe that trying out different solutions for setting $\lambda_0$ should be considered as a potential improvement of the DRLB. 

But this should be considered only after we've trained the model for an appropriate time period, as suggested by Wu et al. (2018). The graph below shows that the DQN was making sensible steps in adjusting the $\lambda$ parameter by increasing for the first 10 steps. Once the agent exhausted its budget though, the DQN actions obviously cannot be evaluated properly.

In [ ]:
# Plot the lambda parameters in the last training episode
ax = last_train_episode['ctl_lambda'].head(50).plot()
ax.set_xlabel("Time step t")
ax.set_ylabel("Lambda parameter at time step t")

# 7. References 

Altman E. (1999): „Constrained Markov decision processes“, CRC Press, Vol. 7.  

Dantzig G. 1957. Discrete-variable extremum problems. Operations research 5, 2 (1957), 266–288.

Han Cai, Kan Ren, Weinan Zhang, Kleanthis Malialis, Jun Wang, Yong Yu, and Defeng Guo. 2017. Real-Time Bidding by Reinforcement Learning in Display Advertising. In Proceedings of the Tenth ACM International Conference on Web
Search and Data Mining. ACM, 661–670.

Singh A. (2019): „Reinforcement Learning : Markov-Decision Process (Part 1)“, [online] https://towardsdatascience.com/introduction-to-reinforcement-learning-markov-decision-process-44c533ebf8da [08.12.2020].  

Stekolshchik R. (2020): „A pair of interrelated neural networks in Deep Q-Network“, [online] https://towardsdatascience.com/a-pair-of-interrelated-neural-networks-in-dqn-f0f58e09b3c4 [09.12.2020].

Richard S. and Andrew G. (2018): "Reinforcement Learning", The MIT Press,pp.1-6. 

Mnih V., Kavukcuoglu K., Silver D., Rusu A.A., Veness J.  Bellemare M., Graves A., Riedmiller M.A., Fidjeland A., Ostrovski G., Petersen S., Beattie C., Sadik A., Antonoglou I., King H., Kumaran D., Wierstra D., Legg S., Hassabis D. (2015): "Human-level control through deep reinforcement learning", Nature 518, 7540, pp. 529–533.

Reza R.A, Yingqian Z., Murat F. et al., A State Aggregation Approach for Solving Knapsack Problem with Deep Reinforcement Learning,Proceedings of Machine Learning Research 129:81–96, 2020

Wu D., Chen X., Yang X., et al. (2018): „Budget Constrained Bidding by model-free Reinforcement Learning in Display Advertising“, Association for Computing Machinery, 27, pp. 1443–1451.  

Zhang W., Yuan S., and Wang J. (2014): „Optimal Real-Time Bidding for Display Advertising“ Association for Computing Machinery, 20, pp. 1077–1086. 

Zhang W., Yuan S., Wang J. (2014): "Real-Time Bidding Benchmarking with iPinYou Dataset", arXiv:1407.7073 [cs.GT]. 

Zhang W., Ren K., and Wang J (2016): "Optimal real-time bidding frameworks discussion", arXiv:1602.01007. 

Zhou Y., Chakrabarty D., and Lukose R. (2008): „Budget constrained bidding in keyword auctions and online knapsack problems“, International Workshop on Internet and Network Economics, Springer, pp. 566–576.  


